In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
zip_path = "/content/drive/MyDrive/archive.zip"
extract_path = "/content/fashion-pattern-images"

In [13]:
import zipfile
import os

# Create target directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")  # Change to `extract_path` if needed

# Optional: Check if extraction succeeded
if os.path.exists(extract_path):
    print(f" Extracted successfully to: {extract_path}")
else:
    print(" Extraction failed or folder name doesn't match.")

 Extracted successfully to: /content/fashion-pattern-images


In [14]:
import os
import shutil
import random

# Paths
original_dataset_dir = '/content/fashion-pattern-images'  # your extracted folder
base_dir = '/content/data'  # base path for train/val
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split ratio
split_ratio = 0.8  # 80% train, 20% val

# Split each class
for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

    images = os.listdir(class_path)
    random.shuffle(images)
    split_idx = int(len(images) * split_ratio)

    for image in images[:split_idx]:
        src = os.path.join(class_path, image)
        dst = os.path.join(train_dir, class_name, image)
        shutil.copyfile(src, dst)

    for image in images[split_idx:]:
        src = os.path.join(class_path, image)
        dst = os.path.join(val_dir, class_name, image)
        shutil.copyfile(src, dst)

print(" Dataset successfully split into train/ and val/")


 Dataset successfully split into train/ and val/


In [15]:
import os
from collections import Counter

dataset_dir = '/content/data/train'
class_counts = Counter()

for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        image_count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        class_counts[class_name] = image_count

for class_name, count in sorted(class_counts.items()):
    print(f"{class_name}: {count} images")


argyle: 441 images
camouflage: 455 images
checked: 450 images
dot: 488 images
floral: 464 images
geometric: 443 images
gradient: 448 images
graphic: 446 images
houndstooth: 448 images
leopard: 448 images
lettering: 441 images
muji: 484 images
paisley: 448 images
snake_skin: 449 images
snow_flake: 467 images
stripe: 476 images
tropical: 451 images
zebra: 507 images
zigzag: 456 images


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'data/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 8710 images belonging to 19 classes.
Found 2188 images belonging to 19 classes.


In [17]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input

base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(19, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/tmp/ipython-input-17-2110589417.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)
]

In [19]:
!mkdir -p "/content/drive/MyDrive/fabric_model"

In [20]:
!ls best_model.keras

ls: cannot access 'best_model.keras': No such file or directory


In [21]:
model_save_path = '/content/drive/MyDrive/fabric_model/best_model.keras'

In [22]:
model.save(model_save_path)
print("✅ Model saved to Google Drive successfully.")

✅ Model saved to Google Drive successfully.


In [24]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/fabric_model/best_model.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 24/273 ━━━━━━━━━━━━━━━━━━━━ 2:06 509ms/step - accuracy: 0.1021 - loss: 3.1931

KeyboardInterrupt: 

In [ ]:
model.save(model_save_path)
print("✅ Model saved to Google Drive successfully.")

✅ Model saved to Google Drive successfully.


In [7]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load your trained model
model = load_model("/content/drive/MyDrive/fabric_model/best_model.keras")

# Class labels
class_labels = [
    "argyle", "camouflage", "checked", "dot", "floral", "geometric",
    "gradient", "graphic", "houndstooth", "leopard", "lettering",
    "muji", "paisley", "snake_skin", "snow_flake", "stripe", "tropical",
    "zebra", "zigzag"
]

# Load and preprocess image
img = image.load_img("/content/paisely1.jpeg", target_size=(224, 224))  # Make sure size matches training
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)[0]  # shape: (19,)

# Print confidence scores for each class
print("\n📊 Confidence for all fabric patterns:")
for label, prob in zip(class_labels, prediction):
    print(f"{label:15s}: {prob * 100:.2f}%")

# Print top prediction
predicted_index = int(np.argmax(prediction))
print("\n✅ Most likely class:", class_labels[predicted_index])
print("🔍 Confidence:", round(prediction[predicted_index] * 100, 2), "%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

📊 Confidence for all fabric patterns:
argyle         : 0.00%
camouflage     : 0.00%
checked        : 0.00%
dot            : 0.00%
floral         : 0.76%
geometric      : 9.59%
gradient       : 0.00%
graphic        : 14.10%
houndstooth    : 0.00%
leopard        : 0.00%
lettering      : 0.52%
muji           : 0.00%
paisley        : 14.59%
snake_skin     : 0.00%
snow_flake     : 4.58%
stripe         : 0.00%
tropical       : 55.84%
zebra          : 0.00%
zigzag         : 0.01%

✅ Most likely class: tropical
🔍 Confidence: 55.84 %


In [10]:
model.trainable = True
# Optionally freeze the first few layers if needed:
for layer in model.layers[:100]:
    layer.trainable = False

# Recompile with a lower learning rate
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Assumes `model` is your trained model and `val_generator` is your validation data generator

# Get true labels and predictions
val_generator.reset()
pred_probs = model.predict(val_generator, verbose=1)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_generator.classes
class_labels = list(val_generator.class_indices.keys())  # Get class label names

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

NameError: name 'val_generator' is not defined